# Machine Learning Primer

Welcome! In this notebook we're going to give you a crash course in a very popular type of application: __machine learning for computer vision__. We'll be working with two widely-used datasets, __MNIST__ and __CIFAR-10__, and we'll use __convolutional neural networks__ to perform __image classification__ on these datasets (in like less than 30 minutes!).

Before we get started, know that image classification builds upon a __lot__ of other topics, including:

- __data science__: how to manipulate and gain insights from data
- __machine learning__: how to use algorithms that can learn to perform tasks from data (rather than just rules)
- __deep learning__: like machine learning, but deeper (lol)
- __computer vision__: enabling computers to see (i.e. understand things about images)

While we can't cover all of these topics in detail in one notebook, we recommend that you check out the Jupyter notebooks on [this page](https://cufctl.github.io/creative-inquiry/) (scroll to the bottom). You can run all of these notebooks out-of-the-box on [Google Colaboratory](https://colab.research.google.com/). In the meantime, we'll try to give you what you need to know to get an image classifier up and running quickly.

## Getting Started

The following code block will import all of the Python modules that you need for this notebook. The easiest way to run this notebook is to use Google Colaboratory, otherwise you will need to have your own Python environment (Anaconda is a great way to do this on your own machine).

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.datasets import mnist
from keras.datasets import cifar10

## Image Classification

Here's the problem: we have a __dataset__ of images -- images of handwritten digits, or everyday objects, or things that may or may not be hot dogs, and so on -- and we want to create a system that can __classify__ each image in the dataset according to its label. So with handwritten digits, the label is the digit (0 to 9). Here are the steps we'll take:

1. Split the dataset into a __training set__ and __testing set__
2. Create a __convolutional neural network (CNN)__ to perform the classification
3. __Train__ the network using the training set
4. __Evaluate__ the network using the test set

At the end of this process we'll have a single value ranging from 0 to 1, the __test accuracy__. It will tell us, _how well did the network classify unseen data after being trained on similar data?_ It is one of the most basic ways to determine how well a machine learning algorithm learned to do something from data.

Before we do all of this, let's talk a little bit about CNNs.

## Convolutions Demystified 

You may have already learned about convolution in another domain, such as math or electrical engineering. You may associate convolution with integrals, and great pain. Fortunately for you, we will not need integrals to use convolution with images. To do convolution on an image, we take a weight matrix and slide it (or "convolve" it) across the image, left to right, top to bottom, like this:

<img src="https://cufctl.github.io/creative-inquiry/assets/img/convolution.gif" alt="convolution" width=400/>

The weight matrix has many names, but we'll call it a __filter__ or __kernel__. At each location in the image, we compute a __weighted sum__ of the pixels in the given window, using the kernel elements as weights. At the end of it we get an "output image", where each "pixel" is the weighted sum centered on that pixel. So in the figure above, the blue square is the input image, the moving green square is the kernel, and the red square is the output of the convolution. Notice that we didn't slide the kernel over pixels where it would have been out of bounds, which means that the output image is smaller. We also usually use kernels that are square and have odd width and height, so that the kernel is centered on the output pixel.

We said that the output is an "image", but what does it represent, and can we even view it as an image? After all, the output is a weighted sum, and the weights can be anything, so we could get output values that are negative, or very large, or not even whole numbers! How about this: let's write some code that performs a convolution on an image and then tries to visualize the output.

## Elements of CNNs

Now that we have a working definition of convolution we can focus on how to incorporate them into our neural network. In particular, there are three new elements that we'll need in order to make convolution work in a neural network: the __convolutional layer__, the __ReLU activation__, and the __pooling layer__.

### Convolutional Layer

This figure shows what a convolutional layer does:

<img src="http://cs231n.github.io/assets/cnn/depthcol.jpeg" alt="depth-column" width=300/>

The red square is the input image. We say that an image has a __width__, __height__, and __depth__. The height and width are intuitive -- in this figure they are 32x32 -- but you may not be used to thinking of an image as having "depth". So far we've only worked with grayscale images, where each pixel has a single "gray" value. In a color image, however, each pixel has three components: red, green, and blue. This type of color image is called an __RGB image__, and the components are called __channels__. Now we can stack these channels so that the image is actually 32x32x3. This means that our kernel -- the small dark red square inside the image -- will also have a depth of 3, in addition to whatever it's width and height are.

The blue cube, which we call the __output volume__, represents the neurons in the output layer. The kernel will be the weight matrix of the convolutional layer, that is, it will represent the connections from the input layer to each neuron in the output layer. There are two key aspects to understand here. First, each neuron in the ouput volume is connected to only a small region of input pixels, rather than all of the input pixels as in a fully-connected layer. Second, every neuron is sharing the same weight matrix! Can you see how this structure is addressing exactly the problems we mentioned before? Now we are focusing on neighboring regions of pixels instead of whole images, and we're accounting for the fact that certain patterns could occur anywhere in the image.

However there's one more thing that we kind of ignored: why is the output a _cube_?! Shouldn't it just be a square? The answer is yes, it would be a square, _if we were only using one kernel_. When we use convolutional layers, we almost always have several kernels in each layer. Each kernel slides across the input image in exactly the same way, but because the kernels will be initialized differently, the idea is that they will each learn a different pattern in the image. One kernel might learn a vertical edge, another kernel a horizontal edge, a third kernel a diagonal edge, and so on. See the blue circles forming a column in the output volume? We call that a __depth column__ -- it represents a group of neurons that are looking at the same region in an image, but through different kernels.

So how big is the output volume? The depth is equal to the number of filters -- every filter produces an activation map. The weight and height depend on a number of factors, which we can summarize in the following equations:

$$W_o = (W_i - F + 2 P) / S + 1$$
$$H_o = (H_i - F + 2 P) / S + 1$$

Where $(W_i, H_i)$ is the input size, $F$ is the width/height of the filter (which we assume is square), $P$ is the amount of __zero padding__, and $S$ is the __stride__. Zero padding refers to adding a border of zeros around the input image, and the stride refers to how far the kernel steps when it slides across the input. We almost always use $F = 2 P + 1$ and $S = 1$, which makes these equations simpler (just a little bit):

$$W_o = W_i$$
$$H_o = W_i$$

In other words, we leave the stride set to 1 (just like the convolution examples from before) and we fix the filter size and zero padding so that the output volume always has the same width and height as the input -- if we use 3x3 kernels then we'll add one layer of zeros, if we use 5x5 kernels then we'll add two layers of zeros, and so on. It's just simpler that way.

### ReLU Activation

<img src="http://cs231n.github.io/assets/nn1/relu.jpeg" alt="relu" width=300/>

So far we've mainly use the sigmoid activation, but there's a new activation function in town that's both simpler and better. It's called the __rectified linear unit (ReLU)__:

$$ReLU(x) = max(0, x)$$

How is it better? Well, Alex Krizhevsky (et. al.) found that their network trained much more quickly when they used ReLU instead of sigmoid. It wasn't that the computations were faster, but that the network didn't need to be trained for as many iterations -- it converged more quickly. We'll use this activation after each convolutional layer, and when we add a few fully-connected layers to the end of our network, we'll use ReLU there too.

### Pooling Layer

<img src="http://cs231n.github.io/assets/cnn/maxpool.jpeg" alt="max-pooling" width=400/>

A pooling layer is essentially a "downsampling" layer -- it breaks the input image into chunks and takes only one value from each chunk. This layer doesn't have any neurons, it just has two settings: (1) the size of the chunks and (2) how to select a value from each chunk. The figure above shows a 2x2 __max-pooling layer__, because each chunk is 2x2 and the maximum value is taken from each chunk. There are other types of pooling layers, like min-pooling and mean-pooling, but max-pooling tends to give the best results, and with good reason. Remember that the output of a convolution is a map that represents what the convolution kernel is "paying attention" to, so by taking only the highest values in each region we are taking the most important features that were computed by the convolutional layer.

We typically use convolutional layers and pooling layers in pairs: the convolutional layer produces an activation map with the same size as the input, and then the pooling layer downsamples the activation map by halving the width and height. At each step, the output volume becomes thinner and deeper, until eventually we can flatten it into a vector and use fully-connected layers at the end of the network.

In [ ]:
# first lets load our data from the keras API
(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [ ]:
# explore some random images from the training set
f, axarr = plt.subplots(4,4, figsize=(8,8))

for i in range(4):
  for j in range(4):
    r = np.random.randint(0, x_train.shape[0], 1)[0]
    img = np.squeeze(x_train[r])
    axarr[i,j].imshow(img, cmap="Greys_r")
    
    axarr[i,j].set_title(str(y_train[r]))
    axarr[i,j].get_xaxis().set_visible(False)
    axarr[i,j].get_yaxis().set_visible(False)    

plt.show()

In [ ]:
# normalize the data between 0 and 1 (255 is the max pixel value)
x_train = np.divide(x_train, 255.0)
x_test = np.divide(x_test, 255.0)

# add an extra dimension to account for a 'channel' (RGB has 3 channels, greyscale has 1)
# the data is currently [n, 28, 28] but we want [n, 28, 28, 1] to work with convolutions
x_train = np.expand_dims(x_train, -1) #-1 just means add an extra dimension to the last axis!
x_test = np.expand_dims(x_test, -1)

# create one hot vectors for labels
# e.g.
# 3 -> [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
# create a basic convolutional neural network
cnn = keras.models.Sequential() 

cnn.add(keras.layers.Conv2D(64, (3,3), padding="same", activation="relu", input_shape=(28,28,1)))
cnn.add(keras.layers.MaxPooling2D(2, 2))

cnn.add(keras.layers.Conv2D(128, (3,3), padding="same", activation="relu"))
cnn.add(keras.layers.MaxPooling2D(2, 2))

cnn.add(keras.layers.Conv2D(128, (3,3), padding="same", activation="relu"))
cnn.add(keras.layers.Flatten())

cnn.add(keras.layers.Dense(1024, activation="relu"))
cnn.add(keras.layers.Dense(10, activation="softmax"))

cnn.summary()

In [ ]:
# compile the model
cnn.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["accuracy"])

# train the model for two epochs, using a batch size of 256
history = cnn.fit(x=x_train, y=y_train, batch_size=256, epochs=5, validation_split=0.1)

In [ ]:
# evaluate the model
score = cnn.evaluate(x=x_test, y=y_test)

print("test loss:     %g" % score[0])
print("test accuracy: %g" % score[1])


In [ ]:
# TODO: Load CIFAR10 from the keras API


In [ ]:
# TODO: Just run this command, I'll leave the code for you to make things easier
f, axarr = plt.subplots(4,4, figsize=(8,8))

for i in range(4):
  for j in range(4):
    r = np.random.randint(0, x_train.shape[0], 1)[0]
    img = np.squeeze(x_train[r])
    axarr[i,j].imshow(img)
    
    axarr[i,j].set_title(str(y_train[r]))
    axarr[i,j].get_xaxis().set_visible(False)
    axarr[i,j].get_yaxis().set_visible(False)    

plt.show()

In [ ]:
# TODO: normalize the data between 0 and 1 (255 is the max pixel value)


# TODO: create one hot vectors for labels
# e.g.
# 3 -> [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


In [ ]:
# TODO: create a basic convolutional neural network
# HINT: MNIST is grayscale, so we just use 1 channel at the beginning...
#       CIFAR10 is RGB, i.e. 3 channels... so make a change at the beginning!
# IDEAS: add more layers! try 'batch normalization' to see if that helps!
# Googling these ideas to increase accuracy will help you win 


In [ ]:
# compile the model
cnn.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["accuracy"])

# train the model for two epochs, using a batch size of 256
history = cnn.fit(x=x_train, y=y_train, batch_size=256, epochs=5, validation_split=0.1)